# 1. TFLite模型生成
## 1.1 预备工作
### 安装程序运行必备的库

In [8]:
%pip install tflite-model-maker

Note: you may need to restart the kernel to use updated packages.


### 导入相关的库

In [9]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

## 1.2 模型训练
### 获取数据 

In [10]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

228827136/228813984 [==============================] - 3s 0us/step


### 加载数据集，并将数据集分为训练数据和测试数据

In [11]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

2024-06-14 09:04:56.558888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /workspaces/mobileLearning/.conda/lib/python3.8/site-packages/cv2/../../lib64:
2024-06-14 09:04:56.558944: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-14 09:04:56.558967: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-4963a3): /proc/driver/nvidia/version does not exist
2024-06-14 09:04:56.573355: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Load image with size: 3670, num_label: 5, labels: daisy, dandelion, roses, sunflowers, tulips.


### 训练Tensorflow模型

In [12]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


2024-06-14 09:05:10.372107: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-06-14 09:05:10.684513: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-06-14 09:05:10.759440: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-06-14 09:05:10.816948: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.
2024-06-14 09:05:10.902673: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 38535168 exceeds 10% of free system memory.


103/103 [==============================] - 101s 957ms/step - loss: 0.8714 - accuracy: 0.7670
Epoch 2/5
103/103 [==============================] - 102s 990ms/step - loss: 0.6527 - accuracy: 0.8996
Epoch 3/5
103/103 [==============================] - 101s 974ms/step - loss: 0.6191 - accuracy: 0.9172
Epoch 4/5
103/103 [==============================] - 100s 971ms/step - loss: 0.5985 - accuracy: 0.9266
Epoch 5/5
103/103 [==============================] - 98s 953ms/step - loss: 0.5848 - accuracy: 0.9345


### 评估模型

In [13]:
loss, accuracy = model.evaluate(test_data)

12/12 [==============================] - 13s 865ms/step - loss: 0.6333 - accuracy: 0.9046


### 导出Tensorflow Lite模型

In [14]:
model.export(export_dir='.')

2024-06-14 09:16:00.042454: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpimb1mkl9/assets


INFO:tensorflow:Assets written to: /tmp/tmpimb1mkl9/assets
2024-06-14 09:16:04.051502: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-06-14 09:16:04.051632: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-06-14 09:16:04.079628: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 913 nodes (656), 923 edges (664), time = 15.723ms.
  function_optimizer: function_optimizer did nothing. time = 0.004ms.

/workspaces/mobileLearning/.conda/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2024-06-14 09:16:04.795365: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored

INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpxahgrpsb/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpxahgrpsb/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite
